<a href="https://colab.research.google.com/github/adrianacupp/NLP_with_Transformers/blob/main/Relation_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#extract dataset directly from kaggle

In [12]:
# 1. Read the kaggle API token to interact with your kaggle account

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adracupp","key":"e74d1222705292f7c525d7e2a435b34f"}'}

In [13]:
#2. Series of commands to set-up for download

!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw-r--r-- 1 root root 64 Aug  6 08:17 kaggle.json
/content


In [14]:
#3. giving rw access (if 401-nathorized)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#4. Sanity check if able to access kaggle
!kaggle datasets list

ref                                                    title                                    size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  --------------------------------------  -----  -------------------  -------------  ---------  ---------------  
waqi786/cats-dataset                                   🐾 Cats Dataset                            6KB  2024-07-31 06:12:49           1688         33  1.0              
priyamchoksi/credit-card-transactions-dataset          Credit Card Transactions Dataset        145MB  2024-07-23 00:31:47           1451         30  1.0              
abhayayare/employee-data-simulation-it-industry        Employee Data Simulation: IT Industry     4KB  2024-07-21 15:26:17           1919         33  1.0              
patricklford/global-ev-sales-2010-2024                 Global EV Sales: 2010-2024               83KB  2024-07-19 13:40:22           2521         35  1.0             

In [19]:
!kaggle datasets download -d daishinkan002/new-york-times-relation-extraction-dataset

Dataset URL: https://www.kaggle.com/datasets/daishinkan002/new-york-times-relation-extraction-dataset
License(s): unknown
new-york-times-relation-extraction-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
!pwd


/content


In [22]:
!ls -lha


total 9.1M
drwxr-xr-x 1 root root 4.0K Aug  6 08:21 .
drwxr-xr-x 1 root root 4.0K Aug  6 07:36 ..
drwxr-xr-x 4 root root 4.0K Aug  2 13:24 .config
drwx------ 6 root root 4.0K Aug  6 08:14 drive
drwxr-xr-x 2 root root 4.0K Aug  6 08:13 .ipynb_checkpoints
-rw-r--r-- 1 root root   64 Aug  6 08:17 kaggle.json
-rw-r--r-- 1 root root 9.1M Jul 31  2021 new-york-times-relation-extraction-dataset.zip
drwxr-xr-x 1 root root 4.0K Aug  2 13:24 sample_data


In [23]:
!unzip new-york-times-relation-extraction-dataset.zip


Archive:  new-york-times-relation-extraction-dataset.zip
  inflating: dataset/test.json       
  inflating: dataset/train.json      
  inflating: dataset/valid.json      


In [27]:
# Function to extract relations from a JSON object
def extract_relations(json_line):
    relations = []
    sent_text = json_line['sentText']
    for relation in json_line['relationMentions']:
        em1_text = relation['em1Text']
        em2_text = relation['em2Text']
        label = relation['label']
        relations.append((sent_text, em1_text, em2_text, label))
    return relations

In [32]:
import json

# Define the file path
input_file = 'dataset/train.json'

# Initialize a list to hold the data
data = []

# Initialize a counter for the number of lines processed
count = 0

# Read the JSON file
with open(input_file, 'r') as file:
    for line in file:
        json_line = json.loads(line)  # Load JSON object from the line
        relations = extract_relations(json_line)  # Extract relations
        data.extend(relations)  # Add extracted relations to the data list
        count += 1  # Increment the counter

In [33]:
# Print the number of lines processed and the first few extracted relations
print(f"Processed {count} lines.")
print(f"First few extracted relations: {data[:5]}")

Processed 56196 lines.
First few extracted relations: [('Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'Annandale-on-Hudson', 'Bard College', '/location/location/contains'), ('North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'North Carolina', 'Greensboro', '/location/location/contains'), ("It will be the final movie credited to Debra Hill , a film producer and native of Haddonfield , who produced '' Halloween '' and was considered a pioneering woman in film .", 'Debra Hill', 'Haddonfield', '/people/person/place_of_birth'), ("In a 3-0 victory over the Crew on Saturday in Columbus , Ohio , goalkeeper Zach Wells stopped Kyle Martino 's penalty kick , only the third unsuccessful penalty in 20 attempts in M.L.S. this season .", 'Ohio', 'Columbus', '/location/location/contains'), ("The United States ambassador to Mexico , Tony Garza , said in a statement that he had directed the American Consulate in Nuev